In [ ]:
import CNN_lib
import CNN_utilities
import fig_lib 
import CNN_machine

import numpy as np
import pickle

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

# GENERAL PARAMETERS

In [ ]:

checkpoint_path = "C:/PROJECTS\emboendo\CNN\python-machine-learning\emboendo_ML\embo_CNN_lab\_static/checkpoints.h5"
pkl_train_p='C:\PROJECTS\emboendo\CNN/pikles/processed_train_d.pkl'  

number_data_ls= [i for i in range(70, 530, 51)]
#number_data_ls= [i for i in range(70, 530, 400)]
check_points_p = [checkpoint_path+'_'+str(n)+'.h5' for n in number_data_ls]
models_perf_d={'model'+'_'+str(n):{} for n in number_data_ls}

test_size=0.2
epochs=9

print(len(number_data_ls),number_data_ls)
print(models_perf_d)


## DATA LOAD

In [ ]:
with open(pkl_train_p, 'rb') as pikle_file:
    save_trains= pickle.load(pikle_file)

Xx_train,Yy_train,rcs =  save_trains[0],save_trains[1],save_trains[2]
max_frm_n, HEIGHT, WIDTH=np.shape(Xx_train)[1],np.shape(Xx_train)[2],np.shape(Xx_train)[3]
input_shape=(max_frm_n, HEIGHT, WIDTH, 1)

## MAIN DATA SHUFFLE

In [ ]:
Xx_train,Yy_train,recs=CNN_lib.shuffle(Xx_train,Yy_train,rcs)

## DATA SPLIT

In [ ]:
test_size=0.2
X_train_spl, X_eval_spl, Y_train_spl, Y_eval_spl = train_test_split(Xx_train, Yy_train, test_size=test_size, shuffle=False) #,random_state=42)

recs_train =recs[0:len(Y_train_spl)]
recs_eval =recs[len(Y_train_spl)::]

## DATA AUGMENTATION

In [ ]:
number_flip_aug_1,number_flip_aug_0 = [],[]

for nn in number_data_ls:

    zeros_count = np.sum(Y_train_spl[0:nn] == 0)
    number_flip_aug_0.append(round(zeros_count*0.07))
    
    ones_count= np.sum(Y_train_spl[0:nn] == 1)
    number_flip_aug_1.append(ones_count)

    print('data:',nn,'zeros:',zeros_count, '--> 7 % :',round(zeros_count*0.07),'ones:',ones_count)

In [ ]:
X_train , Y_train = [] , []

for nn,aug_1,aug_0 in zip(number_data_ls,number_flip_aug_1,number_flip_aug_0):

    aug_X_train_1,aug_Y_train_1,aug_train_recs_1=CNN_lib.main_aug_f(aug_1,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=1)
    aug_X_train_0,aug_Y_train_0,aug_train_recs_0=CNN_lib.main_aug_f(aug_0,X_train_spl[0:nn],Y_train_spl[0:nn],recs[0:nn],label=0)

    X_train_n = np.concatenate((X_train_spl[0:nn], aug_X_train_1,aug_X_train_0), axis=0)
    Y_train_n = np.concatenate((Y_train_spl[0:nn], aug_Y_train_1, aug_Y_train_0))

    recs_n = recs_train[0:nn]+aug_train_recs_1+aug_train_recs_0

    X_train.append(X_train_n)
    Y_train.append(Y_train_n)
     

# CNN

In [ ]:
models=[CNN_machine.Gen_Model(input_shape,pp,nn) for nn,pp in zip(number_data_ls,check_points_p)]

In [ ]:
for mdl,X,Y in zip(models,X_train,Y_train):mdl.train_model(X,Y,X_eval_spl,Y_eval_spl, epochs)

In [ ]:
#for mdl in models:CNN_lib.plot_train_eval(mdl.history,epochs)

In [ ]:
for mdl in models: CNN_lib.plot_roc_curve(mdl.fpr_val,mdl.tpr_val,mdl.roc_auc)

In [ ]:
AUC_ls=[mdl.roc_auc for mdl in models]
number_points=[mdl.tr_pnt for mdl in models]

plt.plot(number_points, AUC_ls, marker='o', linestyle='-', color='b', label='AUC')
plt.xlabel('Número de Puntos')
plt.ylabel('AUC')
plt.title('AUC en función del Número de Puntos')
plt.show()